# Protocolo MPC  SD-in-the-Head

En este notebook, vamos a explicar el protocolo de computación multipartita enfocado en corroborar que una solución $x$ para el problema de decodificación del síndrome es válida. Una solución en este caso se considera válida si tiene un peso de Hamming menor a un valor $w$ y es solución donde $ y^T = Hx^T$.

Para esto, primero vamos a representar las propiedades de la solución $x$ por medio de polinomios y una igualdad entre estos.

## Representación del SDP mediante Polinomios

Para validar la solución al problema de Decodificación del Síndrome (SDP) sin revelar información sensible, se definen cuatro polinomios: $S(X)$, $Q(X)$, $P(X)$, y $F(X)$. Los tres primeros están relacionados con la solución $x$, mientras que $F(X)$ es un polinomio público.

La validación se basa en la siguiente relación polinómica:

$$
S(X) \cdot Q(X) = P(X) \cdot F(X)
$$

Esta igualdad captura las propiedades algebraicas del vector $x$, asegurando la corrección de la solución.

### Definición de Polinomios

- **$S(X)$:**  
  Se obtiene mediante interpolación de Lagrange sobre $x$.  
  $$
  S(f_i) = x_i \quad \text{para cada } i \in [1 : n]
  $$  
  Grado: $\deg(S) \leq n - 1$.

- **$Q(X)$:**  
  Contiene las posiciones no nulas de $x$.  
  $$
  Q(X) = \prod_{i \in E} (X - f_i)
  $$  
  Grado: $\deg(Q) = w$, donde $w$ es el peso de Hamming de $x$.

- **$F(X)$:**  
  Se anula en todos los $f_i$.  
  $$
  F(X) = \prod_{i=1}^{n} (X - f_i)
  $$  
  Grado: $\deg(F) = n$.

- **$P(X)$:**  
  Relaciona los términos anteriores.  
  $$
  P(X) = \frac{S(X) \cdot Q(X)}{F(X)}
  $$  
  Grado: $\deg(P) \leq w - 1$.

### Significado de $f_i$

Cada $f_i$ representa un elemento del campo finito $\mathbb{F}_q$, asociado de manera única a un índice $i$ del vector $x$. Esta correspondencia permite evaluar los polinomios en los puntos $f_i$, estableciendo una conexión algebraica directa entre los índices del vector $x$ y los puntos del campo finito.

### Propiedades y Relación de Grados

El lado izquierdo, $S(X) \cdot Q(X)$, se anula en todos los puntos $f_i$, ya que:
- $S(f_i) = 0$ si $x_i = 0$.
- $Q(f_i) = 0$ si $x_i \neq 0$.

El polinomio $F(X)$ también se anula en todos los $f_i$, garantizando que ambos lados compartan los mismos puntos de anulación. Para mantener el equilibrio, el grado de $P(X)$ se ajusta a $w - 1$, logrando que la igualdad se cumpla.

Finalmente, $Q(X)$ puede anularse en $w$ puntos, asegurando que $S(X)$ no se anule en más de $w$ puntos. Esto limita el peso de Hamming del vector $x$ a $w$, como exige el esquema.


### Polinomios

Procedemos a calcular cada uno de los polinomios asociados a un vector $x$ que es solución al problema de decodificación del síndrome. 

In [1]:
# Definir parámetros
definir_campo_q = 2  # Campo finito para el problema de decodificación del síndrome
F_q = GF(definir_campo_q)  # Campo finito de orden q
n = 6  # Longitud del vector de error
k = 3  # Longitud del vector de mensaje
m = n - k  # Número de filas de la matriz de paridad H

# Campo finito para los polinomios (debe ser mayor que F_q)
definir_campo_p = 17  # Campo finito para los polinomios, debe ser mayor que F_q y mayor que n
F_p = GF(definir_campo_p)  # Campo finito de orden p

# Definir el vector x en el campo F_q de longitud n
x = vector(F_q, [0, 0, 0, 0, 1, 1])  # Ejemplo con valores específicos

# Asociar los primeros n elementos del campo F_p con las coordenadas de x
f = [F_p(i) for i in range(n)]  # f_i para i en [1:n]

# Polinomio S: Interpolación de Lagrange usando los puntos (f_i, x_i)
R = F_p['X']
S = R.lagrange_polynomial([(f[i], x[i]) for i in range(n)])

# Polinomio Q: Producto de (X - f_i) para i en el subconjunto de índices con valor x_i = 1
X = R.gen()
E = [i for i in range(n) if x[i] != 0]  # Subconjunto de índices no nulos de x
Q = prod([(X - f[i]) for i in E])

# Polinomio F: Polinomio de "desaparición" para todos los f_i
F = prod([(X - f[i]) for i in range(n)])

# Polinomio P: Definido como P = S * Q / F
P = (S * Q).quo_rem(F)[0]  # División polinomial asegurando que F divide a S * Q sin residuo

# Evaluar los polinomios S y Q en los puntos f
evaluaciones_S = [S(f_i) for f_i in f]
evaluaciones_Q = [Q(f_i) for f_i in f]

# Imprimir los polinomios
def imprimir_polinomios():
    print("Polinomio S (Interpolación de Lagrange):")
    print(S)
    print("\nEvaluaciones de S en f:")
    print(evaluaciones_S)
    print("\nSubconjunto E:")
    print(E)
    print("\nPolinomio Q (Producto de raíces para índices no nulos de x):")
    print(Q)
    print("\nEvaluaciones de Q en f:")
    print(evaluaciones_Q)
    print("\nPolinomio F (Vanishing Polynomial):")
    print(F)
    print("\nPolinomio P (Resultado de S * Q / F):")
    print(P)
    
    print("\nEvaluación de Polinomios")
    print(f"\nS*Q==P*F: {S*Q==P*F}")

imprimir_polinomios()

Polinomio S (Interpolación de Lagrange):
13*X^5 + 11*X^4 + 10*X

Evaluaciones de S en f:
[0, 0, 0, 0, 1, 1]

Subconjunto E:
[4, 5]

Polinomio Q (Producto de raíces para índices no nulos de x):
X^2 + 8*X + 3

Evaluaciones de Q en f:
[3, 12, 6, 2, 0, 0]

Polinomio F (Vanishing Polynomial):
X^6 + 2*X^5 + 13*X^3 + 2*X^2 + 16*X

Polinomio P (Resultado de S * Q / F):
13*X + 4

Evaluación de Polinomios

S*Q==P*F: True


In [2]:
# Función para evaluar S * Q == P * F en puntos aleatorios del campo F_p
def verificar_igualdad_en_puntos_aleatorios(S, Q, P, F, intentos):
    # Definir el polinomio en el campo F_p
    R_p = F_p['X']
    S_p = R_p(S)
    Q_p = R_p(Q)
    P_p = R_p(P)
    F_p_poly = R_p(F)

    # Realizar las evaluaciones en puntos aleatorios del campo F_p
    for _ in range(intentos):
        punto_aleatorio = F_p.random_element()
        valor_SQ = S_p(punto_aleatorio) * Q_p(punto_aleatorio)
        valor_PF = P_p(punto_aleatorio) * F_p_poly(punto_aleatorio)
        if valor_SQ != valor_PF:
            print(f"Desigualdad encontrada en el punto {punto_aleatorio}: S * Q = {valor_SQ}, P * F = {valor_PF}")
            return False
    print("Los polinomios S * Q y P * F son iguales en todos los puntos evaluados.")
    return True

# Evaluar la igualdad en puntos aleatorios
evaluaciones_aleatorias = 10  # Número de intentos deseado
verificar_igualdad_en_puntos_aleatorios(S, Q, P, F, evaluaciones_aleatorias)

Los polinomios S * Q y P * F son iguales en todos los puntos evaluados.


True

## Construcción del protocolo de MPC para SDitH

Con lo planteado previamente, ahora deseamos construir el protocolo MPC para el problema de decodificación del síndrome.La idea de este protocolo es demostrar que se posee una solución para el problema de decodificación del síndrome que cumple la propiedad del peso de Hamming y para esto se usa el Protocolo MPC para verificación multiplicativa. 

1. Las partes obtienen un valor aleatorio $\epsilon \in \mathbb{F}$ (proporcionado por el verificador en el paradigma MPCitH).

2. Las partes calculan localmente $[\alpha] = \epsilon \cdot [Q(r)] + [a]$ y $[\beta] = [S(r)] + [b]$.

3. Las partes transmiten $[\alpha]$ y $[\beta]$ para obtener públicamente $\alpha$ y $\beta$.

4. Las partes calculan localmente:

   $$
   [v] = \epsilon \cdot [F \cdot P(r)] - [c] + \alpha \cdot [b] + \beta \cdot [a] - \alpha \cdot \beta
   $$

5. Las partes transmiten $[v]$ para obtener públicamente $v$.

6. Las partes aceptan si $v = 0$ y rechazan en caso contrario.


Para hace esto, es se usan las propiedades de compartición de secretos y en este caso el esquema aditivo.

In [3]:
# Definir parámetros
definir_campo_q = 2  # Campo finito para el problema de decodificación del síndrome
F_q = GF(definir_campo_q)  # Campo finito de orden q
n = 6  # Longitud del vector de error
k = 3  # Longitud del vector de mensaje
m = n - k  # Número de filas de la matriz de paridad H

# Campo finito para los polinomios (debe ser mayor que F_q)
definir_campo_p = 17  # Campo finito para los polinomios, debe ser mayor que F_q y mayor que n
F_p = GF(definir_campo_p)  # Campo finito de orden p

# Definir el vector x en el campo F_q de longitud n
x = vector(F_q, [0, 0, 0, 0, 1, 1])  # Ejemplo con valores específicos

# Asociar los primeros n elementos del campo F_p con las coordenadas de x
f = [F_p(i) for i in range(n)]  # f_i para i en [1:n]

# Polinomio S: Interpolación de Lagrange usando los puntos (f_i, x_i)
R = F_p['X']
S = R.lagrange_polynomial([(f[i], x[i]) for i in range(n)])

# Polinomio Q: Producto de (X - f_i) para i en el subconjunto de índices con valor x_i = 1
X = R.gen()
E = [i for i in range(n) if x[i] != 0]  # Subconjunto de índices no nulos de x
Q = prod([(X - f[i]) for i in E])

# Polinomio F: Polinomio de "desaparición" para todos los f_i
F = prod([(X - f[i]) for i in range(n)])

# Polinomio P: Definido como P = S * Q / F
P = (S * Q).quo_rem(F)[0]  # División polinomial asegurando que F divide a S * Q sin residuo

# Evaluar los polinomios S y Q en los puntos f
evaluaciones_S = [S(f_i) for f_i in f]
evaluaciones_Q = [Q(f_i) for f_i in f]

# Imprimir los polinomios
def imprimir_polinomios():
    print("Polinomio S (Interpolación de Lagrange):")
    print(S)
    print("\nEvaluaciones de S en f:")
    print(evaluaciones_S)
    print("\nSubconjunto E:")
    print(E)
    print("\nPolinomio Q (Producto de raíces para índices no nulos de x):")
    print(Q)
    print("\nEvaluaciones de Q en f:")
    print(evaluaciones_Q)
    print("\nPolinomio F (Vanishing Polynomial):")
    print(F)
    print("\nPolinomio P (Resultado de S * Q / F):")
    print(P)
    
    print("\nEvaluación de Polinomios")
    print(f"\nS*Q==P*F: {S*Q==P*F}")

imprimir_polinomios()

Polinomio S (Interpolación de Lagrange):
13*X^5 + 11*X^4 + 10*X

Evaluaciones de S en f:
[0, 0, 0, 0, 1, 1]

Subconjunto E:
[4, 5]

Polinomio Q (Producto de raíces para índices no nulos de x):
X^2 + 8*X + 3

Evaluaciones de Q en f:
[3, 12, 6, 2, 0, 0]

Polinomio F (Vanishing Polynomial):
X^6 + 2*X^5 + 13*X^3 + 2*X^2 + 16*X

Polinomio P (Resultado de S * Q / F):
13*X + 4

Evaluación de Polinomios

S*Q==P*F: True


In [4]:
# Dividimos los polinomios S, Q y P en partes
n_parts = 5  # Número de partes

def dividir_polinomio_en_partes(polinomio, n_parts, campo):
    coeficientes = polinomio.coefficients(sparse=False)
    grado = polinomio.degree()
    partes = [[] for _ in range(n_parts)]  # Crear listas para cada parte

    for i in range(grado + 1):
        # Generamos aleatoriamente las primeras (n_parts - 1) partes en el campo
        partial_shares = [campo.random_element() for _ in range(n_parts - 1)]
        # Calculamos la última parte de manera que la suma de todas las partes sea igual al coeficiente original
        last_share = coeficientes[i] - sum(partial_shares)
        # Añadimos la última parte a la lista de partes
        partial_shares.append(last_share)
        # Distribuimos cada parte correspondiente entre las listas de partes de los participantes
        for j in range(n_parts):
            partes[j].append(partial_shares[j])
        
        # Imprimir las partes generadas para el coeficiente actual
        print(f"Coeficiente de x^{i} ({coeficientes[i]}): Partes -> {partial_shares}")

    return partes

# Función para generar los polinomios a partir de las partes
def construir_polinomios_de_partes(shares, n_parts, X):
    polinomios = []
    for j in range(n_parts):
        part_poly = sum(shares[j][i] * X^i for i in range(len(shares[j])))
        polinomios.append(part_poly)
    return polinomios

# Dividimos los polinomios S, Q y P en partes
print("\nDividiendo el polinomio S en partes:")
shares_S = dividir_polinomio_en_partes(S, n_parts, F_p)

print("\nDividiendo el polinomio Q en partes:")
shares_Q = dividir_polinomio_en_partes(Q, n_parts, F_p)

print("\nDividiendo el polinomio P en partes:")
shares_P = dividir_polinomio_en_partes(P, n_parts, F_p)

# Construimos los polinomios para cada una de las partes
def imprimir_polinomios_partes():
    partes_S_polinomios = construir_polinomios_de_partes(shares_S, n_parts, X)
    partes_Q_polinomios = construir_polinomios_de_partes(shares_Q, n_parts, X)
    partes_P_polinomios = construir_polinomios_de_partes(shares_P, n_parts, X)

    print("\nPolinomio que recibe cada parte para el polinomio S:")
    for j, part_poly in enumerate(partes_S_polinomios):
        print(f"Parte {j + 1} de S: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio Q:")
    for j, part_poly in enumerate(partes_Q_polinomios):
        print(f"Parte {j + 1} de Q: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio P:")
    for j, part_poly in enumerate(partes_P_polinomios):
        print(f"Parte {j + 1} de P: {part_poly}")

imprimir_polinomios_partes()

# Función para generar valores aleatorios en el campo
def generar_valores_aleatorios(campo, n_parts):
    valores = [campo.random_element() for _ in range(n_parts)]
    print(f"Valores aleatorios generados: {valores}")
    return valores

# Generar valores aleatorios para a_shares y b_shares
print("\nGenerando partes para a y b:")
a_shares = generar_valores_aleatorios(F_p, n_parts)
b_shares = generar_valores_aleatorios(F_p, n_parts)

# Calcular el producto c = a * b, y luego compartir el valor c
c = sum(a_shares) * sum(b_shares)
print(f"\nValor de c (producto de a y b): {c}")

# Generar partes para c_shares
c_shares = generar_valores_aleatorios(F_p, n_parts)
c_sum = sum(c_shares)
adjustment = c - c_sum
print(f"\nSuma de las partes de c antes del ajuste: {c_sum}")
print(f"Ajuste necesario para c: {adjustment}")

# Ajustar la primera parte de c_shares para garantizar que sum(c_shares) == c
c_shares[0] += adjustment
print(f"Partes ajustadas de c: {c_shares}")

print("\nResumen de lo que recibe cada parte:")
for i in range(n_parts):
    print(f"Parte {i + 1} recibe:")
    print(f"  - Coeficientes de S: {shares_S[i]}")
    print(f"  - Coeficientes de Q: {shares_Q[i]}")
    print(f"  - Coeficientes de P: {shares_P[i]}")
    print(f"  - Parte de a: {a_shares[i]}")
    print(f"  - Parte de b: {b_shares[i]}")
    print(f"  - Parte de c: {c_shares[i]}")


Dividiendo el polinomio S en partes:
Coeficiente de x^0 (0): Partes -> [5, 9, 0, 15, 5]
Coeficiente de x^1 (10): Partes -> [13, 16, 5, 15, 12]
Coeficiente de x^2 (0): Partes -> [0, 11, 14, 13, 13]
Coeficiente de x^3 (0): Partes -> [5, 10, 14, 9, 13]
Coeficiente de x^4 (11): Partes -> [8, 15, 10, 8, 4]
Coeficiente de x^5 (13): Partes -> [13, 2, 1, 11, 3]

Dividiendo el polinomio Q en partes:
Coeficiente de x^0 (3): Partes -> [5, 7, 13, 7, 5]
Coeficiente de x^1 (8): Partes -> [10, 7, 16, 15, 11]
Coeficiente de x^2 (1): Partes -> [0, 11, 3, 14, 7]

Dividiendo el polinomio P en partes:
Coeficiente de x^0 (4): Partes -> [4, 2, 7, 14, 11]
Coeficiente de x^1 (13): Partes -> [12, 15, 13, 1, 6]

Polinomio que recibe cada parte para el polinomio S:
Parte 1 de S: 13*X^5 + 8*X^4 + 5*X^3 + 13*X + 5
Parte 2 de S: 2*X^5 + 15*X^4 + 10*X^3 + 11*X^2 + 16*X + 9
Parte 3 de S: X^5 + 10*X^4 + 14*X^3 + 14*X^2 + 5*X
Parte 4 de S: 11*X^5 + 8*X^4 + 9*X^3 + 13*X^2 + 15*X + 15
Parte 5 de S: 3*X^5 + 4*X^4 + 13*X^

In [5]:
# Implementación del protocolo MPC
# Paso 1: Obtener un valor aleatorio epsilon en el campo finito
def obtener_valor_aleatorio(campo):
    return campo.random_element()

epsilon = obtener_valor_aleatorio(F_p)
print(f"\nValor aleatorio epsilon: {epsilon}")

# Paso 2: Evaluar los polinomios S, Q, y P en un punto aleatorio r del campo finito
def evaluar_polinomios_en_punto(polinomios, r):
    return [part(r) for part in polinomios]

r = obtener_valor_aleatorio(F_p)
print(f"Valor aleatorio {r}")
print(f"Valuación en punto aleatorio {S(r)*Q(r)==P(r)*F(r)}")

evaluaciones_S_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_S, n_parts, X), r)
evaluaciones_Q_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_Q, n_parts, X), r)
evaluaciones_P_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_P, n_parts, X), r)

# Paso 3: Calcular [alpha] y [beta] para cada parte
def calcular_alpha_beta(epsilon, evaluaciones_Q, evaluaciones_S, a_shares, b_shares):
    comparticion_alpha = [epsilon * evaluaciones_Q[j] + a_shares[j] for j in range(n_parts)]
    comparticion_beta = [evaluaciones_S[j] + b_shares[j] for j in range(n_parts)]
    return comparticion_alpha, comparticion_beta

comparticion_alpha, comparticion_beta = calcular_alpha_beta(epsilon, evaluaciones_Q_en_r, evaluaciones_S_en_r, a_shares, b_shares)

# Paso 4: Transmitir [alpha] y [beta] para obtener alpha y beta
def reconstruir_valor(comparticiones):
    return sum(comparticiones)

alpha = reconstruir_valor(comparticion_alpha)
beta = reconstruir_valor(comparticion_beta)
print(f"\nValor de alpha: {alpha}")
print(f"Valor de beta: {beta}")

# Paso 5: Calcular [v] para cada parte
# Calcular c como la multiplicación de la suma de a_shares y la suma de b_shares, luego dividir c en partes aleatorias
def calcular_v(epsilon, evaluaciones_P, a_shares, b_shares, alpha, beta, c_shares):
    comparticion_v = []
    for j in range(n_parts):
        if j == 0:
            # Para el primer elemento, restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j] - alpha * beta
        else:
            # Para los demás elementos, no restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j]
        comparticion_v.append(v_j)
    return comparticion_v

comparticion_v = calcular_v(epsilon, evaluaciones_P_en_r, a_shares, b_shares, alpha, beta, c_shares)

# Paso 5: Transmitir [v] para obtener v
v = reconstruir_valor(comparticion_v)
print(f"\nValor de v: {v}")

# Paso 7: Verificar si v es igual a 0
def verificar_v(v):
    if v == 0:
        print("Las partes aceptan: v = 0")
    else:
        print(f"Las partes rechazan: v = {v}")

verificar_v(v)


Valor aleatorio epsilon: 9
Valor aleatorio 16
Valuación en punto aleatorio True

Valor de alpha: 13
Valor de beta: 0

Valor de v: 0
Las partes aceptan: v = 0
